In [ ]:
import pandas as pd
import numpy as np
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from upsetplot import UpSet
from matplotlib import pyplot
import matplotlib

In [ ]:
import os

if not os.path.exists("Figures"):
    os.mkdir("Figures")

In [ ]:
ddf = dd.read_parquet("../hpc-datasets/mutations.parquet", engine="pyarrow")
ddf_all_commits = dd.read_parquet("../hpc-datasets/all_commits.parquet", engine="pyarrow")
ddf_commits = dd.read_parquet("../hpc-datasets/commits_ext.parquet", engine="pyarrow")
ddf_repoinfos = dd.read_parquet("../hpc-datasets/repo_infos.parquet", engine="pyarrow")

In [ ]:
ddf

In [ ]:
ddf_repoinfos = dd.read_parquet("../hpc-datasets/repo_infos.parquet", engine="pyarrow")

In [ ]:
df_repoinfos = pd.read_parquet("../hpc-datasets/repo_infos.parquet", engine="pyarrow")

In [ ]:
ddf_repoinfos['max_file_depth'] = ddf_repoinfos['max_file_depth']+1
df_repoinfos['max_file_depth'] = df_repoinfos['max_file_depth']+1

In [ ]:
df_github = pd.read_csv("../datasets/github_data_07_03_2024.csv",keep_default_na=False)
df_github["repo_id"] = df_github["owner.name"].str.lower() + "_" + df_github["name"].str.lower()
df_github = df_github[~df_github['ignored']]
df_github = df_github.set_index('repo_id')

In [ ]:
import yaml
linguist_langs = yaml.safe_load(open("../datasets/linguist/languages.yml", "r"))

In [ ]:
linguist_types = set(linguist_langs[x]['type'] for x in linguist_langs)
linguist_types

In [ ]:
# ddf_all_commits = ddf_all_commits.set_index("repo_id")
# ddf_commits = ddf_commits.set_index("commit_id") 
# df_repoinfos = df_repoinfos.set_index("repo_id")
# ddf = ddf.set_index("repo_id")

In [ ]:
ddf_commits.columns

In [ ]:
df_github

In [ ]:
df_repoinfos

In [ ]:
gh_info_cols = ['lang', 'stargazers_count', 'owner.type']
df_repoinfos = df_repoinfos.merge(df_github[gh_info_cols], on='repo_id')
# df_repoinfos = ddf_repoinfos.compute()

# Computing of difference in depth between commits
Copmute the difference in depth between consecutive commits, save this into a dataframe and store a extended dataset to disk. 

In [ ]:
df_repoinfos.loc['conradkleinespel_rooster']

In [ ]:
tmp_all_commits = ddf_all_commits[ddf_all_commits['repo_id'] == 'conradkleinespel_rooster'].compute()

In [ ]:
tmp_all_commits

In [ ]:
tmp_commits = ddf_commits[ddf_commits['repo_id'] == 'conradkleinespel_rooster'].compute()

In [ ]:
tmp_commits = tmp_commits.sort_values('commit_index', ascending=True)

In [ ]:
tmp_muts = ddf[ddf['repo_id'] == 'conradkleinespel_rooster'].compute()

In [ ]:
tmp_muts[tmp_muts['old_path'].str.contains('safe_vec.rs') | tmp_muts['new_path'].str.contains('safe_vec.rs')]

In [ ]:
tmp_commits[tmp_commits['commit_id'] == '53e03787430b7f135bb86bc495fe712f776f7d32']

In [ ]:
tmp_commits[tmp_commits['commit_id'] == 'cbf0680a48986857a97424b6b0f49ae02fcdf147']

In [ ]:
tmp_commits[tmp_commits['commit_id'] == '3f12c511db10118d5c320e5af7b0bb0941048ba2']

In [ ]:
tmp_commits.to_csv('test.csv')

In [ ]:
ddf_commits[(ddf_commits['max_file_depth_diff'] > 0) & (ddf_commits['n_added'] == 0) & (ddf_commits['n_moved'] == 0)].compute()

# Investigating the data

In [ ]:
# What percentage of commits have file structural changes
ddf_commits.shape[0].compute()/ddf_all_commits.shape[0].compute()

In [ ]:
(ddf_commits.groupby('repo_id').count().compute()/ddf_all_commits.groupby('repo_id').count().compute()).mean()

In [ ]:
ddf_commits['n_struc_changes'].describe().compute()

In [ ]:
ddf_commits[['n_added', 'n_deleted', 'n_moved', 'n_renamed']].describe().compute()

In [ ]:
ddf_commits[['message', 'n_added', 'n_deleted', 'n_moved', 'n_renamed']].describe().compute()

In [ ]:
fig_data = ddf_commits[['message', 'n_added', 'n_deleted', 'n_moved', 'n_renamed']].compute()

In [ ]:
(fig_data[fig_data['n_added']> 0]['n_added'].describe(),
fig_data[fig_data['n_deleted']> 0]['n_deleted'].describe(),
fig_data[fig_data['n_moved']> 0]['n_moved'].describe(),
fig_data[fig_data['n_renamed']> 0]['n_renamed'].describe())

In [ ]:
(np.log10(fig_data[fig_data['n_added']> 0]['n_added']).describe(),
np.log10(fig_data[fig_data['n_deleted']> 0]['n_deleted']).describe(),
np.log10(fig_data[fig_data['n_moved']> 0]['n_moved']).describe(),
np.log10(fig_data[fig_data['n_renamed']> 0]['n_renamed']).describe())

In [ ]:
# Step 1: Filter the data
def plot_struc_changes(col: str):
    filtered_data = fig_data[fig_data[col] > 0][col]
    
    # Step 2: Bin the data using logarithmic bins for integers
    bin_edges = np.unique(np.logspace(np.log10(1), np.log10(filtered_data.max()), num=50).astype(int))
    
    # Compute the counts for each bin
    counts, bin_edges = np.histogram(filtered_data, bins=bin_edges)
    
    # Prepare data for plotting
    bin_labels = [f"{int(edge1)}-{int(edge2-1)}" for edge1, edge2 in zip(bin_edges[:-1], bin_edges[1:])]
    
    # Step 3: Create the bar plot
    fig = go.Figure(go.Bar(x=bin_labels, y=counts, name="Counts"))
    fig.update_layout(
        title="Bar Plot of Binned Additions",
        xaxis_title="Bins of Number of Additions",
        yaxis_title="Counts",
        xaxis_type="category",  # Binned categories
        yaxis_type="log"  # Logarithmic scale to handle wide range of counts
    )
    fig.show()
plot_struc_changes('n_added')


In [ ]:
plot_struc_changes('n_deleted')


In [ ]:
plot_struc_changes('n_moved')


In [ ]:
plot_struc_changes('n_renamed')


In [ ]:
filtered_data[filtered_data['n_deleted'] > filtered_data['n_deleted'].quantile(0.9999)]

In [ ]:

np.random.seed(42)  # For reproducibility
subsample_size = 500_000
subsample_add = np.random.choice(fig_data[fig_data['n_added']> 0]['n_added'], size=subsample_size, replace=False)
subsample_del = np.random.choice(fig_data[fig_data['n_deleted']> 0]['n_deleted'], size=subsample_size, replace=False)
subsample_mov = np.random.choice(fig_data[fig_data['n_moved']> 0]['n_moved'], size=subsample_size, replace=False)
subsample_ren = np.random.choice(fig_data[fig_data['n_renamed']> 0]['n_renamed'], size=subsample_size, replace=False)

fig = go.Figure()
fig.add_trace(go.Violin(y=np.log10(subsample_add)))
fig.add_trace(go.Violin(y=np.log10(subsample_del)))
fig.add_trace(go.Violin(y=np.log10(subsample_mov)))
fig.add_trace(go.Violin(y=np.log10(subsample_ren)))
# fig.update_yaxes(type='log')
fig.update_traces(box_visible=True, meanline_visible=True,)
fig.write_image('tmp.pdf')
fig.show()

In [ ]:
bar_data_add = fig_data[fig_data['n_added']> 0]['n_added'].value_counts()
bar_data_del = fig_data[fig_data['n_deleted']> 0]['n_deleted'].value_counts()
bar_data_mov = fig_data[fig_data['n_moved']> 0]['n_moved'].value_counts()
bar_data_ren = fig_data[fig_data['n_renamed']> 0]['n_renamed'].value_counts()

fig = go.Figure()
fig.add_trace(go.Bar(y=bar_data_add))
fig.add_trace(go.Bar(y=bar_data_del))
fig.add_trace(go.Bar(y=bar_data_mov))
fig.add_trace(go.Bar(y=bar_data_ren))
fig.write_image('tmp.pdf')

In [ ]:
(ddf_commits[['n_added', 'n_deleted', 'n_moved', 'n_renamed']] > 0).sum().compute()/ddf_commits.shape[0].compute()

In [ ]:
upsetData = pd.DataFrame({
    'add': ddf_commits['n_added'] > 0,
    'delete': ddf_commits['n_deleted'] > 0,
    'move': ddf_commits['n_moved'] > 0,
    'rename': ddf_commits['n_renamed'] > 0
})

In [ ]:
upset_grouped_df = upsetData.groupby(['add','delete','move','rename']).size()

In [ ]:
matplotlib.rcParams["font.size"] = 6
matplotlib.rcParams["axes.facecolor"] = "white"

ax_dict = UpSet(upset_grouped_df, show_percentages="{:.2%}", sort_by="cardinality", facecolor="#c81919").plot()
pyplot.savefig("Figures/upset_file_structure_changes.pdf")

In [ ]:
ddf_commits.head()

In [ ]:
%%time
ddf_repoinfos.groupby('repo_id').count().compute()

In [ ]:
%%time
ddf.groupby('repo_id').count().compute()

In [ ]:
%%time
ddf_commits.groupby('repo_id').count().compute()

In [ ]:
%%time
ddf_all_commits.groupby('repo_id').count().compute()

In [ ]:
%%time
# merged_commits_ddf = ddf_all_commits.join(ddf_commits, rsuffix="_commit", how="left").groupby('repo_id')

In [ ]:
%%time
# merged_commits_ddf.count().compute()

# General Statistics

In [ ]:
df_repoinfos

## Number of contributors

In [ ]:
df_repoinfos["no_contributors"].describe()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(y=df_repoinfos["no_contributors"]))

fig.update_layout(title='Number of Contributors',
                  yaxis_title='Number of Contributors')

fig.show()


In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(y=np.log10(df_repoinfos["no_contributors"])))

fig.update_layout(title='Number of Contributors',
                  yaxis_title='Number of Contributors')

fig.show()


## Nuber of commits

In [ ]:
df_repoinfos["no_commits"].describe()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(y=df_repoinfos["no_commits"]))

fig.update_layout(title='Number of Commits',
                  yaxis_title='Number of Commits')

fig.show()


In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(y=np.log10(df_repoinfos["no_commits"])))

fig.update_layout(title='Number of Commits',
                  yaxis_title='Number of Commits')

fig.show()


In [ ]:
df_repoinfos[df_repoinfos["no_commits"] > 250000]

In [ ]:
df_repoinfos[df_repoinfos["no_commits"] == 1]

## Max file depth

In [ ]:
df_repoinfos["max_file_depth"].describe()

In [ ]:
df_repoinfos["max_file_depth"].describe()

In [ ]:
print(df_repoinfos["max_file_depth"].describe().to_latex())

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(y=df_repoinfos["max_file_depth"]))

fig.update_layout(title='Max File Depth',
                  yaxis_title='Max File Depth')

fig.show()
# fig.write_image('Figures/max_file_depth.pdf')

In [ ]:
outlier_data.index

In [ ]:
df_repoinfos["max_file_depth"].mean()

In [ ]:
fig = go.Figure()

# Filtered version
tmp_box_data = df_repoinfos["max_file_depth"]
thresh = 40

box_data = tmp_box_data[tmp_box_data <= thresh]
outlier_data = tmp_box_data[tmp_box_data > thresh]

boxplot_trace = go.Box(y=box_data, name='Data')
outlier_trace = go.Scatter(
    x=['Data']*len(outlier_data),
    y=outlier_data,
    mode='markers+text',
    marker=dict(color='red', size=5),
    text=outlier_data,
    textposition='middle right',
)

# Create the figure
fig = make_subplots(rows=2, cols=1, vertical_spacing = 0.02, row_heights=[0.25, 0.75])
fig.add_trace(boxplot_trace, row=2, col=1)

# Add outlier scatter traces to the figure
fig.add_trace(outlier_trace, row=1, col=1)

# Hide the legend
fig.update_layout(showlegend=False, height=500, width=1000, template="ggplot2")

# Hide axis labels in the upper subplot
fig.update_xaxes(showticklabels=False, showgrid=False, row=1, col=1)
fig.update_yaxes(title_text='', row=1, col=1)
fig.update_xaxes(showticklabels=False, showgrid=False, row=2, col=1)
fig.update_traces(marker_color='rgb(200, 25, 25)')

# fig.update_yaxes(title_text='Maximum file structure depth', row=2, col=1)

# Add x-axis title
# fig.update_xaxes(title_text='Maximum Project Depth', row=2, col=1)

# Synchronize y-axis ranges between subplots
# fig.update_yaxes(range=[35, outlier_data['max_file_depth'].max()+15], row=1, col=1)
# fig.update_yaxes(range=[box_data['max_file_depth'].min()-5, 35], row=2, col=1)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
# Show the plot
fig.show()
fig.write_image('Figures/max_file_depth.pdf')


In [ ]:
tmp_box_data.describe(percentiles=[0.25,0.5,0.75,0.90,0.95,.99,0.999])

In [ ]:
tmp_box_data.describe(percentiles=[0.25,0.5,0.75,0.90,0.95,.99,0.999])

In [ ]:
df_repoinfos["max_file_depth"].describe(percentiles=[0.25,0.5,0.75,0.90,0.95,.99])

In [ ]:
ddf_commits["max_file_depth"].describe(percentiles=[0.25,0.5,0.75,0.90,0.95,.99]).compute()

In [ ]:
ddf_commits['max_file_depth_diff'].describe(percentiles=[0.25,0.5,0.75,0.90,0.95,.99]).compute()

In [ ]:
ddf_commits['max_file_depth_diff'].abs().describe(percentiles=[0.25,0.5,0.75,0.90,0.95,.99]).compute()

In [ ]:
initial_outlier_proj = df_repoinfos[(df_repoinfos['max_file_depth'] >= 16)]
initial_outlier_proj.shape[0]

In [ ]:
initial_outlier_proj

In [ ]:
initial_outlier_proj.groupby(['lang']).count()

In [ ]:
temp_data = ddf_commits[ddf_commits["max_file_depth_diff"].abs() > 0]

In [ ]:
commits_depth_inc = ddf_commits[(ddf_commits["max_file_depth_diff"] > 0) & (~ddf_commits["max_file_depth_diff"].isin([123,50,40]))]
commits_depth_inc["max_file_depth_diff"].describe(percentiles=[0.25,0.5,0.75,0.90,0.95,.99,.999,.9999]).compute()

In [ ]:
commits_depth_inc.compute().groupby('lang')["max_file_depth_diff"].describe(percentiles=[0.25,0.5,0.75,0.90,0.95,.99,.999,.9999])

In [ ]:
fig = px.box(commits_depth_inc["max_file_depth_diff"])
fig.show()

In [ ]:
df_repoinfos[df_repoinfos["max_file_depth"].isin([123,57,49])]

In [ ]:
ddf_commits[ddf_commits["max_file_depth"].isin([123,57,49]) & (ddf_commits["max_file_depth_diff"] > 0)].compute()

In [ ]:
ddf_commits[ddf_commits['commit_id'] == "8afe8378a69a39616daba30b4eafa4b2602e17b2"].compute()

In [ ]:
ddf[ddf["old_file_depth"].isin([122,56,48])].compute()

In [ ]:
commits_depth_dec = ddf_commits[(ddf_commits["max_file_depth_diff"] < 0) & (~ddf_commits["max_file_depth_diff"].isin([-119,-50]))]
commits_depth_dec["max_file_depth_diff"].describe(percentiles=[0.01,0.1,0.25,0.5,0.75]).compute()

In [ ]:
commits_depth_dec.compute().groupby('lang')["max_file_depth_diff"].describe(percentiles=[0.01,0.05,0.1,0.25,0.5,0.75])

In [ ]:
fig = make_subplots(rows=len(df_repoinfos['lang'].unique()), cols=2)

for i, lang in enumerate(df_repoinfos['lang'].unique()):
    lang_commits_depth_inc = commits_depth_inc[commits_depth_inc['lang'] == lang]
    fig.add_trace(
        go.Violin(
            x=[lang] * len(lang_commits_depth_inc["max_file_depth_diff"]), 
            y=lang_commits_depth_inc["max_file_depth_diff"]), 
        row=i+1, col=1
    )
        
    lang_commits_depth_dec = commits_depth_dec[commits_depth_dec['lang'] == lang]
    fig.add_trace(
        go.Violin(
            x=[lang] * len(lang_commits_depth_dec["max_file_depth_diff"]), 
            y=-lang_commits_depth_dec["max_file_depth_diff"]),
        row=i+1, col=2
    )

fig.update_traces(box_visible=False, meanline_visible=False)
fig.update_layout(height=750, template="ggplot2", showlegend=False)

# fig.add_hline(y=12, opacity=1, line_width=2, line_color="black", row=1, col=1)
# fig.add_hline(y=8.32, opacity=1, line_width=2, line_color="black", row=1, col=2)

fig.show()
fig.write_image("Figures/violin_depth_difference.pdf")

In [ ]:
fig = make_subplots(rows=1, cols=2)
# fig = go.Figure()

fig.add_trace(
    go.Violin(
        x=['Depth Increase'] * len(commits_depth_inc), 
        y=commits_depth_inc["max_file_depth_diff"]), 
                        # legendgroup='INC', scalegroup='INC', name='INC',
                        # side='negative'),
    row=1, col=1
)
        
    # lang_commits_depth_dec = commits_depth_dec[commits_depth_dec['lang'] == lang].compute()
fig.add_trace(
    go.Violin(
        x=['Depth Decrease'] * len(commits_depth_dec), 
        y=-commits_depth_dec["max_file_depth_diff"]),
                        # legendgroup='DEC', scalegroup='DEC', name='DEC',
                        # side='positive'),
    row=1, col=2
)

fig.update_traces(box_visible=True, meanline_visible=True)
fig.update_layout(height=750, template="ggplot2", showlegend=False)

fig.add_hline(y=12, opacity=1, line_width=2, line_color="black", row=1, col=1)
fig.add_hline(y=9, opacity=1, line_width=2, line_color="black", row=1, col=2)

fig.update_traces(marker_color="rgb(200,25,25)", row=1, col=1)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()
fig.write_image("Figures/violin_depth_difference.pdf")

In [ ]:
fig = make_subplots(rows=2, cols=1)
# fig = go.Figure()
# for i, lang in enumerate(df_repoinfos['lang'].unique()):
# lang_commits_depth_inc = commits_depth_inc[commits_depth_inc['lang'] == lang].compute()
fig.add_trace(
    go.Box(
        x=commits_depth_inc["lang"], 
        y=commits_depth_inc["max_file_depth_diff"]), 
                        # legendgroup='INC', scalegroup='INC', name='INC',
                        # side='negative'),
    row=1, col=1
)
        
    # lang_commits_depth_dec = commits_depth_dec[commits_depth_dec['lang'] == lang].compute()
fig.add_trace(
    go.Box(
        x=commits_depth_dec["lang"], 
        y=-commits_depth_dec["max_file_depth_diff"]),
                        # legendgroup='DEC', scalegroup='DEC', name='DEC',
                        # side='positive'),
    row=2, col=1
)

fig.update_traces(box_visible=False, meanline_visible=False)
fig.update_layout(height=1024, template="ggplot2", showlegend=False)

# fig.add_hline(y=12, opacity=1, line_width=2, line_color="black", row=1, col=1)
# fig.add_hline(y=8.32, opacity=1, line_width=2, line_color="black", row=1, col=2)

fig.show()
fig.write_image("Figures/violin_pc_depth_difference.pdf")

In [ ]:
fig = px.violin(commits_depth_inc.compute(), x='lang', y="max_file_depth_diff")
fig.update_layout(height=800)
fig.show()

In [ ]:
fig = px.violin(commits_depth_dec.compute(), x='lang', y="max_file_depth_diff")
fig.update_layout(height=800)
fig.show()

In [ ]:
commits_depth_dec.compute().groupby('lang')["max_file_depth_diff"].describe(percentiles=[0.01,0.05,0.1,0.25,0.5,0.75])

In [ ]:
large_increase_commits = ddf_commits[(ddf_commits["max_file_depth_diff"] >= commits_depth_inc["max_file_depth_diff"].quantile(0.99)) & (ddf_commits['commit_index'] != 0)].compute()

In [ ]:
ddf_commits[ddf_commits["max_file_depth_diff"] < 0]["max_file_depth_diff"].describe(percentiles=[0.01,0.1,0.25,0.5,0.75]).compute()

In [ ]:
df_repoinfos

In [ ]:
ddf_commits = ddf_commits.merge(df_repoinfos[['lang']], left_on='repo_id', right_index=True)

In [ ]:
ddf_commits

In [ ]:
large_increase_commits = large_increase_commits

In [ ]:
lic_class = large_increase_commits.merge(df_intro_classified[['repo_id', 'commit_id','Reason 1', 'Reason 2']], on=['repo_id', 'commit_id'], how='left')


In [ ]:
lic_class

In [ ]:
lic_class.groupby(['lang']).count()

In [ ]:
tmp_muts = ddf[ddf['repo_id'].isin(lic_class['repo_id']) & ddf['commit_id'].isin(lic_class['commit_id'])].compute()

In [ ]:
grouped_tmp_muts = tmp_muts.set_index(['repo_id', 'commit_id', 'new_file_depth'])

In [ ]:
tmp_intro_commits = lic_class
tmp_intro_commits['new_file_depth'] = tmp_intro_commits['max_file_depth']-1
tmp_intro_commits = tmp_intro_commits.set_index(['repo_id', 'commit_id', 'new_file_depth'])
tmp_intro_commits

In [ ]:
df_intro_commits = tmp_intro_commits.merge(
    grouped_tmp_muts[['new_path']],
    left_index=True,
    right_index=True,
    how='left'
).reset_index().drop_duplicates(['repo_id', 'commit_id', 'new_file_depth']).rename(columns={'new_path': 'long_path'})
df_intro_commits.head()

In [ ]:
df_intro_commits.to_excel('diff_not_classified.xlsx')

In [ ]:
lic_class = pd.read_excel('diff_with_classified.xlsx')
lic_class.loc[lic_class['Reason 1'].isin(['Node Modules', 'Vendor',"Pods","Python Packages","Carthage"]),"Reason 1"] = "Vendor"

In [ ]:
lic_class.merge(ddf_commits[['repo_id', 'commit_id', 'pct_added', 'pct_deleted']].compute(), on=['repo_id', 'commit_id'])

In [ ]:
lic_class[lic_class['Reason 1'].isna()]

In [ ]:
print(lic_class['Reason 1'].value_counts().to_latex(escape=True))

In [ ]:
lic_class.groupby('commit_index')['Reason 1'].value_counts().head(50)

In [ ]:
tmp_data = lic_class.groupby('Reason 1')['commit_id'].count()
fig = px.box(tmp_data)
fig.show()

In [ ]:
commits_depth_dec = commits_depth_dec.merge(df_repoinfos[['lang']], on='repo_id')
decrease_99 = commits_depth_dec[commits_depth_dec['max_file_depth_diff'] <= -8.320000].compute()

pd.concat([decrease_99,lic_class]).to_excel('depth_dec_unclassified.xlsx')

In [ ]:
df_intro_classified['Reason 1'].value_counts()

In [ ]:
len(df_intro_classified['Reason 1'])

In [ ]:
tuples1 = [tuple(x) for x in lic_class[['repo_id', 'commit_id']].to_records(index=False)]
tuples2 = [tuple(x) for x in df_intro_classified[['repo_id', 'commit_id']].to_records(index=False)]

# Find the subset of tuples that are present in both DataFrames
subset = set(tuples1).intersection(tuples2)
len(subset)

In [ ]:
max_old_file_depths = ddf[ddf['repo_id'].isin(decrease_99['repo_id']) & ddf['commit_id'].isin(decrease_99['commit_id'])].groupby(['repo_id', 'commit_id'])['old_file_depth'].max().compute()

In [ ]:
tmp_del_commits = decrease_99 # pd.read_excel('depth_dec_classified.xlsx')
tmp_del_commits = tmp_del_commits.set_index(['repo_id', 'commit_id'])
tmp_del_commits['old_file_depth'] = max_old_file_depths
tmp_del_commits = tmp_del_commits.reset_index().set_index(['repo_id', 'commit_id', 'old_file_depth'])
tmp_del_commits

In [ ]:
grouped_tmp_muts = ddf[ddf['repo_id'].isin(decrease_99['repo_id']) & ddf['commit_id'].isin(decrease_99['commit_id'])].compute().set_index(['repo_id', 'commit_id', 'old_file_depth'])

In [ ]:
df_tmp_del_commits = tmp_del_commits.merge(
    grouped_tmp_muts[['old_path']],
    left_index=True,
    right_index=True,
    how='left'
).reset_index().drop_duplicates(['repo_id', 'commit_id', 'old_file_depth']).rename(columns={'old_path': 'long_path'})
df_tmp_del_commits.head()

In [ ]:
pd.concat([df_tmp_del_commits,lic_class]).to_excel('depth_dec_unclassified.xlsx')

In [ ]:
print(lic_class.groupby('Reason 1')['lang'].value_counts().to_latex(escape=True, longtable=True))

# Percentage of files changes to total files

In [ ]:
ddf_commits['pct_added'] = dd.Series.round(ddf_commits['n_added']/(ddf_commits['total_files']+1)*100, 1)
ddf_commits['pct_deleted'] = dd.Series.round(ddf_commits['n_deleted']/(ddf_commits['total_files']+ddf_commits['n_deleted'])*100, 1)
ddf_commits['pct_touched'] = dd.Series.round(ddf_commits['n_struc_changes']/(ddf_commits['total_files']+ddf_commits['n_deleted']+1)*100, 1)

In [ ]:
ddf_commits['pct_added'].describe().compute()

In [ ]:
ddf_commits['pct_touched'].describe().compute()

In [ ]:
ddf_commits[ddf_commits['pct_deleted'] > 100].compute()

In [ ]:
fig = px.scatter(ddf_commits.compute(), x='commit_index', y='pct_touched')
fig.write_image('tmp.pdf')

In [ ]:
fig=px.box(ddf_commits['pct_touched'])
fig.write_image('tmp2.pdf')

# End

In [ ]:
# Create a box plot of the filtered data
fig = go.Figure()

fig.add_trace(go.Box(y=temp_data["max_file_depth_diff"].abs()))

# Update layout with titles and axis labels
fig.update_layout(
    title='Max File Depth Diff',
    yaxis_title='Max File Depth Difference'
)

# Show the plot
fig.show()
# fig.write_image('Figures/.pdf')

In [ ]:
temp_data["max_file_depth_diff"].abs().describe(percentiles=[0.25,0.5,0.75,0.90,0.95,.99,.999])

In [ ]:
# outlier_data_2 = ddf[~((ddf['new_file_depth'] >= lower_bound) & (ddf['new_file_depth'] <= upper_bound))].compute()
# outlier_data_2.sort_values('new_file_depth', ascending=False).drop_duplicates(subset='commit_id')

In [ ]:
# outlier_data.sort_values("max_file_depth", ascending=False)

In [ ]:
depth_occurences = df_repoinfos.groupby("max_file_depth")["first_commit"].count()

In [ ]:
import scipy.stats as stats

mean = 3.10
std_dev = 2.05

# Generate x values for the distribution curve
x = np.linspace(0, np.max(depth_occurences), 100)
# Calculate corresponding y values from the normal distribution
y = len(depth_occurences)*stats.norm.pdf(x, loc=mean, scale=std_dev)

In [ ]:
fig = px.bar(depth_occurences)
fig.show()# Add the distribution curve as a line plot

In [ ]:
fig = px.bar(depth_occurences)
fig.show()# Add the distribution curve as a line plot

In [ ]:
# If you want to handle multiple modes and select the first or maximum, you can do:
def select_mode(modes):
    if len(modes) > 1:
        return modes.max()  # or modes.min() or modes.iloc[0] depending on your requirement
    else:
        return modes[0]

depth_occurences_commits = ddf_commits.compute().groupby(['repo_id'])["max_file_depth"].agg(lambda x: select_mode(pd.Series.mode(x)))

In [ ]:
depth_occurences_commits.mean()

In [ ]:
depth_occurences_commits.value_counts()

In [ ]:
fig = px.bar(depth_occurences_commits.value_counts())
fig.update_layout(template='ggplot2', height=300, width=1000, showlegend=False)
fig.update_traces(marker_color='rgb(200,25,25)')
fig.update_xaxes(title="File Tree Depth")
fig.update_yaxes(title="Number of Projects")
fig.show()# Add the distribution curve as a line plot
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.write_image('Figures/average_file_tree_depth.pdf')

In [ ]:
(ddf_repoinfos['no_commits'] == 3).sum().compute()

In [ ]:
# Calculate counts of occurrences for each file depth level
counts = df_repoinfos['max_file_depth'].value_counts()

# Perform the Kolmogorov-Smirnov test on the counts data
mean_counts = counts.mean()
std_dev_counts = counts.std()

statistic, p_value = stats.kstest(counts, 'norm', args=(mean_counts, std_dev_counts))

# Interpret the test result
alpha = 0.05  # Significance level
print(f"Kolmogorov-Smirnov Test on Counts:")
print(f"Test Statistic: {statistic:.4f}")
print(f"P-value: {p_value:.4f}")

if p_value < alpha:
    print("The counts do not follow a normal distribution.")
else:
    print("The counts follow a normal distribution.")

In [ ]:
df_repoinfos[df_repoinfos["max_file_depth"] > 40]

In [ ]:
df_repoinfos["max_file_depth"].quantile(0.99)

In [ ]:
fig = px.scatter(df_repoinfos, x="no_contributors", y="no_commits")
fig.show()


In [ ]:
fig = px.scatter(df_repoinfos, x="stargazers_count", y="no_commits")
fig.show()

In [ ]:
fig = px.scatter(df_repoinfos, x="stargazers_count", y="no_contributors")
fig.show()

In [ ]:
# ddf_all_commits = ddf_all_commits.set_index("commit_id")
# df_repoinfos_first = ddf_repoinfos.merge(ddf_all_commits, left_on="first_commit", right_index=True, how="left")
# df_repoinfos_last = ddf_repoinfos.merge(ddf_all_commits, left_on="last_commit", right_index=True, how="left")

In [ ]:
# df_repoinfos_first.compute()

# Differentiate by Programming Language

In [ ]:
df_repoinfos.head()

In [ ]:
ddf_repoinfos[ddf_repoinfos["lang"] == ''].compute()

In [ ]:
df_github.loc['duo001_evil.js']

In [ ]:
fig = px.box(df_repoinfos, x='lang', y='max_file_depth')
fig.update_xaxes(categoryorder='category ascending')
fig.show()

In [ ]:
# Filtered version for making the graph easier to read
tmp_box_data = df_repoinfos
thresh = 40

box_data = tmp_box_data[tmp_box_data['max_file_depth'] <= thresh]
outlier_data = tmp_box_data[tmp_box_data['max_file_depth'] > thresh]

categories = box_data.sort_values(by='lang')['lang'].unique()
count_per_lang = tmp_box_data.groupby('lang').count()['first_commit']
xlabels = {name: f"{name}<br>({count})" for name, count in count_per_lang.items()}
print (xlabels)

# Create the figure
fig = make_subplots(rows=2, cols=1, vertical_spacing = 0.02, row_heights=[0.25, 0.75])

for lang in categories:
    cat_data = box_data[box_data['lang'] == lang]
    fig.add_trace(go.Box(
        x=[xlabels[lang]] * cat_data.shape[0], 
        y=cat_data['max_file_depth']), 
                  row=2, 
                  col=1
    )

outlier_traces = []
for lang in categories:
    lang_outlier_data = outlier_data[outlier_data['lang'] == lang]
    if not lang_outlier_data.empty:
        trace = go.Scatter(
            x=[lang] * len(lang_outlier_data),
            y=lang_outlier_data['max_file_depth'],
            mode='markers+text',
            name=lang,
            marker=dict(color='red', size=5),
            text=lang_outlier_data['max_file_depth'],
            textposition='middle right',
        )
        outlier_traces.append(trace)
    else:
        # Create an empty trace with category name
        trace = go.Scatter(
            x=[lang],
            y=[None],  # Provide empty y-values
            mode='markers',
            name=lang,
            marker=dict(color='red', size=0)  # Set size to 0 for invisible markers
        )
        outlier_traces.append(trace)

fig.update_xaxes(categoryorder='category ascending', row=2, col=1)

# fig.add_trace(outliers_trace, row=1, col=1)
# Add outlier scatter traces to the figure
for trace in outlier_traces:
    fig.add_trace(trace, row=1, col=1)

# Update layout for clarity (optional)
# fig.update_layout(title='Maximum File Tree Depth per Programming Language', yaxis_title='Value')

# Hide the legend
fig.update_layout(showlegend=False, height=500, template="ggplot2", width=1000)

# Hide axis labels in the upper subplot
fig.update_xaxes(showticklabels=False, showgrid=False, row=1, col=1)
fig.update_yaxes(title_text='', row=1, col=1)
fig.update_traces(marker_color="rgb(200,25,25)")

# Synchronize y-axis ranges between subplots
# fig.update_yaxes(range=[35, outlier_data['max_file_depth'].max()+15], row=1, col=1)
# fig.update_yaxes(range=[box_data['max_file_depth'].min()-5, 35], row=2, col=1)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

# Show the plot
fig.show()
fig.write_image('Figures/max_file_depth_per_lang.pdf')

## Analyse outliers per programming language

In [ ]:
pcs_in_data = df_repoinfos.groupby('lang')["max_file_depth"].describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99])

In [ ]:
pcs_in_data

In [ ]:
df_repoinfos_with_tresh = (
    df_repoinfos
    .reset_index()
    .merge(pcs_in_data[['99%']], how='left', left_on='lang', right_on='lang')
    .set_index('repo_id')
)

In [ ]:
df_repoinfos_above_thresh = (
    df_repoinfos_with_tresh[df_repoinfos_with_tresh['max_file_depth'] >= df_repoinfos_with_tresh['99%']]
        .rename(columns={ "max_file_depth": "proj_max_file_depth" })
)

In [ ]:
df_repoinfos_above_thresh

In [ ]:
df_repoinfos_above_thresh['lang'].value_counts()

In [ ]:
outlier_projects = df_repoinfos_above_thresh.index.tolist()

Get all the commits that are part of the outlier projects

In [ ]:
outlier_project_commits = ddf_commits[ddf_commits["repo_id"].isin(outlier_projects)]

In [ ]:
outlier_project_commits = outlier_project_commits.merge(df_repoinfos_above_thresh[["proj_max_file_depth", "lang"]], on="repo_id")

In [ ]:
# Remove initial commits, only commits that happen later in the evolution
outlier_project_commits = outlier_project_commits[outlier_project_commits["parents"] != '']

In [ ]:
outlier_project_commits.head()

In [ ]:
df_intro_commits = outlier_project_commits[(outlier_project_commits["max_file_depth"] == outlier_project_commits["proj_max_file_depth"]) & (outlier_project_commits["max_file_depth_diff"] > 0)].compute()

In [ ]:
# TEST

In [ ]:
outlier_project_commits[outlier_project_commits['max_file_depth_diff'] > 0]['max_file_depth_diff'].describe(percentiles=[0.25, 0.75, 0.9, 0.99]).compute()

In [ ]:
outlier_project_commits[outlier_project_commits['max_file_depth_diff'] >= 10.3].compute()

In [ ]:
outlier_project_commits[outlier_project_commits['max_file_depth_diff'] < 0]['max_file_depth_diff'].describe(percentiles=[0.001, 0.01, 0.1, 0.25, 0.75]).compute()

In [ ]:
outlier_project_commits[outlier_project_commits['max_file_depth_diff'] <= -7].compute()

In [ ]:
df_intro_commits

Find the mutations that caused the depth to increase;
Notice, we only store a single path as there could be very very many per commit, but should be enough to determine what happened!

In [ ]:
tmp_muts = ddf[ddf['repo_id'].isin(df_intro_commits['repo_id']) & ddf['commit_id'].isin(df_intro_commits['commit_id'])].compute()

In [ ]:
grouped_tmp_muts = tmp_muts.set_index(['repo_id', 'commit_id', 'new_file_depth'])

In [ ]:
tmp_intro_commits = df_intro_commits
tmp_intro_commits['new_file_depth'] = tmp_intro_commits['max_file_depth']-1
tmp_intro_commits = tmp_intro_commits.set_index(['repo_id', 'commit_id', 'new_file_depth'])
tmp_intro_commits

In [ ]:
df_intro_commits = tmp_intro_commits.merge(
    grouped_tmp_muts[['new_path']],
    left_index=True,
    right_index=True,
    how='left'
).reset_index().drop_duplicates(['repo_id', 'commit_id', 'new_file_depth']).rename(columns={'new_path': 'long_path'})
df_intro_commits.head()

We check that there are no commits where it could not find a mutation with this long of a path. This is to check the bug is gone, which we found late in the process of analysis and we had to retrieve new dataset.

In [ ]:
commits_with_no_path = df_intro_commits[df_intro_commits['long_path'].isna()]
assert(len(commits_with_no_path) == 0)

In [ ]:
df_intro_commits["node_modules"] = df_intro_commits["long_path"].str.contains("node_modules/", case=False)

In [ ]:
df_intro_commits["vendor"] = df_intro_commits["long_path"].str.contains("vendor/", case=False)

In [ ]:
df_intro_commits["pods"] = df_intro_commits["long_path"].str.contains("pods/", case=False)

In [ ]:
df_intro_commits["python-packages"] = df_intro_commits["long_path"].str.contains("site-packages/", case=False) | df_intro_commits["long_path"].str.contains("dist-packages/", case=False)

In [ ]:
df_intro_commits["carthage"] = df_intro_commits["long_path"].str.contains("carthage/", case=False)

In [ ]:
df_intro_commits["tests"] = df_intro_commits["long_path"].str.contains("tests/|test/|Fixtures/", case=False)

In [ ]:
df_intro_classified = pd.read_excel('INTRO_COMMIT_REASON5.xlsx')

In [ ]:
df_intro_commits = df_intro_commits.merge(df_intro_classified[['repo_id', 'commit_id', 'Reason 1', 'Reason 2']],
                                                              how='left',
                                                              on=['repo_id', 'commit_id']
                                                             )

In [ ]:
df_intro_commits

In [ ]:
# Update reasons
df_intro_commits.loc[df_intro_commits["node_modules"]& (df_intro_commits['Reason 1'] == ''), "Reason 1"] = "Node Modules"
df_intro_commits.loc[df_intro_commits["vendor"]& (df_intro_commits['Reason 1'] == ''), "Reason 1"] = "Vendor"
df_intro_commits.loc[df_intro_commits["pods"]& (df_intro_commits['Reason 1'] == ''), "Reason 1"] = "Pods"
df_intro_commits.loc[df_intro_commits["python-packages"]& (df_intro_commits['Reason 1'] == ''), "Reason 1"] = "Python Packages"
df_intro_commits.loc[df_intro_commits["carthage"]& (df_intro_commits['Reason 1'] == ''), "Reason 1"] = "Carthage"
df_intro_commits.loc[df_intro_commits["tests"]& (df_intro_commits['Reason 1'] == ''), "Reason 1"] = "Tests"

In [ ]:
df_intro_commits.to_excel('INTRO_COMMIT_REASON_UNCLASSIFIED.xlsx')

If code is skipped to generate the intro commits with the automated reasons, this is to import the manually classified file:

In [ ]:
df_intro_classified = pd.read_excel('INTRO_COMMIT_REASON5.xlsx')

If the specific package manager is not interesting for the analysis, set all the package manager reasons to "Vendor", which is to denote some kind of package manager is used.

In [ ]:
df_intro_classified.loc[df_intro_classified['Reason 1'].isin(['Node Modules', 'Vendor',"Pods","Python Packages","Carthage"]),"Reason 1"] = "Vendor"

In [ ]:
len(df_intro_classified)

In [ ]:
df_intro_classified['Reason 1'].value_counts(),df_intro_classified['Reason 1'].value_counts()/len(df_intro_classified)*100

In [ ]:
df_intro_classified["node_modules"].sum()

In [ ]:
df_intro_classified[df_intro_classified["node_modules"]]["lang"].value_counts()

In [ ]:
df_intro_classified[df_intro_classified["node_modules"] & (df_intro_classified["lang"]=="Python")]

In [ ]:
df_intro_classified[df_intro_classified["vendor"]]["lang"].value_counts()

In [ ]:
df_intro_commits[df_intro_commits["vendor"] & (df_intro_commits["lang"]=="Python")]

In [ ]:
df_intro_commits[df_intro_commits["site-packages"]]["lang"].value_counts()

In [ ]:
df_intro_commits[df_intro_commits["site-packages"] & (df_intro_commits["lang"]=="C_C++")]

In [ ]:
df_intro_commits[df_intro_commits["pods"]]["lang"].value_counts()

In [ ]:
df_intro_commits[df_intro_commits["pods"] & (df_intro_commits["lang"]=="Kotlin")]

In [ ]:
df_intro_commits[df_intro_commits["node_modules"] | df_intro_commits["vendor"] | df_intro_commits["site-packages"] | df_intro_commits["pods"]].count()

In [ ]:
bla = df_intro_classified['long_path'].str.split('/')
bla = bla.explode('file_parts')

In [ ]:
bla.value_counts().head(25)

In [ ]:
df_intro_commits_merged["tests"].value_counts()

In [ ]:
df_intro_commits_merged[df_intro_commits_merged["long_path"] == ''].value_counts()

In [ ]:
vendor_filter = df_intro_commits_merged["node_modules"] | df_intro_commits_merged["vendor"] | df_intro_commits_merged["python-packages"] | df_intro_commits_merged["pods"] | df_intro_commits_merged["carthage"]
df_intro_commits_merged[vendor_filter].count()

In [ ]:
df_intro_commits_merged[df_intro_commits_merged["tests"] & ~vendor_filter].count()

In [ ]:
df_intro_commits_merged[~df_intro_commits_merged["tests"] & ~vendor_filter]

In [ ]:
df_intro_commits_merged

In [ ]:
df_intro_classified[df_intro_classified['Reason 1'] == 'Error?'

In [ ]:
fig = px.scatter(df_intro_classified, x='Reason 1', y='max_file_depth_diff')
fig.show()

In [ ]:
depth_dec_classes = pd.read_excel('depth_decrease_classified.xlsx')

In [ ]:
depth_dec_classes = depth_dec_classes[depth_dec_classes['max_file_depth_diff'] < 0]

In [ ]:
fig = px.scatter(depth_dec_classes, x='Reason 1', y='max_file_depth_diff')
fig.show()

In [ ]:
depth_dec_classes['Reason 1'].value_counts()

In [ ]:
print(depth_dec_classes['Reason 1'].value_counts().to_latex())

In [ ]:
depth_dec_classes.shape[0]

In [ ]:
depth_dec_classes[depth_dec_classes['Reason 1'] == "Node Modules"].groupby('lang').count()

In [ ]:

depth_dec_classes['commit_time'] = pd.to_datetime(depth_dec_classes['commit_time'], unit="s")

In [ ]:
go_mod_times = depth_dec_classes[(depth_dec_classes['Reason 1'] == "Go Modules")]
non_go_mod_times = depth_dec_classes[(depth_dec_classes['lang'] == "Go")]
go_mod_plotly = go_mod_times.groupby([pd.Grouper(key='commit_time', freq='1m')]).count()
go_non_mod_plotly = non_go_mod_times.groupby([pd.Grouper(key='commit_time', freq='1m')]).count()

In [ ]:
fig = go.Figure()
# fig = px.scatter(go_mod_times, x='commit_time', y=[1]*len(go_mod_times))
fig.add_vline(pd.to_datetime(1535112000, unit="s"),label=dict(
        text="Go Modules introduced",
        textposition="end",
        font=dict(size=10, color="black"),
        yanchor="bottom",
    ), opacity=1, line_width=2, line_color="black", )
fig.add_vline(pd.to_datetime("26-02-2019", dayfirst=True),
    label=dict(
        text="Go Deps deprecated",
        textposition="end",
        font=dict(size=10, color="black"),
        yanchor="bottom",
    ), opacity=1, line_width=2, line_color="black",  )
# fig.update_traces(marker_color='rgb(200, 25, 25)')
fig.add_trace(go.Scatter(x=go_mod_plotly.index, y=go_mod_plotly['Column2'], mode='lines', name='Go Modules', marker_color='rgb(200, 25, 25)'))
fig.add_trace(go.Scatter(x=go_non_mod_plotly.index, y=go_non_mod_plotly['Column2'], mode='lines', name='Total Go related'))

fig.update_layout(width=800, height=300, template='ggplot2')

fig.update_layout(
    margin=dict(l=20, r=20, t=80, b=20),
)

fig.show()
fig.write_image('Figures/go_modules_intro.pdf')

## Check which are reversed

In [ ]:
long_paths_filter = ddf['old_path'].isin(df_intro_classified['long_path']) & ddf['repo_id'].isin(df_intro_classified['repo_id'])
later_modifications = ddf[long_paths_filter].compute()

In [ ]:
later_modifications

In [ ]:
later_commits = ddf_commits[ddf_commits['commit_id'].isin(later_modifications['commit_id']) & ddf_commits['repo_id'].isin(later_modifications['repo_id']) & (ddf_commits['max_file_depth_diff'] < 0)].compute()

In [ ]:
later_commits

In [ ]:
correction_data = later_commits.merge(later_modifications[["commit_id","old_path","new_path","action"]], on='commit_id').set_index(['repo_id', 'commit_id'])

In [ ]:
correction_data

In [ ]:
intro_with_correction = pd.concat([df_intro_classified.rename(columns={'long_path':'old_path'}), correction_data.reset_index()]).set_index(['repo_id', 'old_path'])

In [ ]:
intro_with_correction.to_excel('INTRO_COMMIT_REASON4_CORRECTIONS.xlsx')

In [ ]:
df_intro_classified = df_intro_classified.set_index(['repo_id', 'commit_id'])

In [ ]:
correction_data

In [ ]:
correction_data

In [ ]:
intro_correction_data = df_intro_classified.reset_index().rename(columns={'long_path':'old_path'}).merge(
    correction_data.reset_index()[['repo_id', 'commit_id', 'action', 'max_file_depth_diff', 'message', 'commit_time', 'old_path']].rename(columns={
        'action': 'correction', 
        'max_file_depth_diff': 'correction_depth_diff',
        'message': 'correction_message',
        'commit_time': 'correction_time',
        'commit_id': 'correction_id'
    }), 
    how='left',
    on=['repo_id', 'old_path']
)
intro_correction_data

In [ ]:
intro_correction_data[intro_correction_data['correction'].isna()]

In [ ]:
intro_correction_data['correction_diff'] = intro_correction_data['max_file_depth_diff'] + intro_correction_data['correction_depth_diff']

In [ ]:
intro_correction_data['correction_diff'].value_counts()

In [ ]:
vendor_commits = intro_correction_data[intro_correction_data['Reason 1'] == "Vendor"]
(vendor_commits['correction_diff'] <= 0).sum()/len(vendor_commits)

In [ ]:
(intro_correction_data['correction_diff'] <= 0).sum()/len(intro_correction_data)

In [ ]:
intro_correction_data.groupby('Reason 1')['correction'].count()/intro_correction_data.groupby('Reason 1')['old_path'].count()

In [ ]:
intro_correction_data[['repo_id', 'old_path', 'commit_id', 'commit_time', 'commit_index','message', 'max_file_depth', 'max_file_depth_diff', 'lang', 
                       'correction', 'correction_id', 'correction_depth_diff', 'correction_message', 'correction_time', 'correction_diff',
                       'Reason 1', 'Reason 2']].to_excel('depth_intro_correction.xlsx')


In [ ]:
fig = px.box(intro_correction_data, x='Reason 1', y='correction_diff')
fig.show()

In [ ]:
fig = px.box(intro_correction_data, x='Reason 1', y='correction_depth_diff')
fig.show()

In [ ]:
intro_correction_data.groupby('Reason 1')['correction_diff'].describe()

In [ ]:
fig = px.scatter(intro_correction_data, x='max_file_depth_diff', y='correction_diff', color='Reason 1')
fig.show()

In [ ]:
fig = px.scatter(intro_correction_data, x='Reason 1', y='correction_depth_diff')
fig.show()

In [ ]:
intro_correction_data['correction_after'] = intro_correction_data['correction_time'] - intro_correction_data['commit_time']

In [ ]:
fig = px.scatter(intro_correction_data, y='correction_diff', x='correction_after')
fig.show()

In [ ]:
intro_correction_data

In [ ]:
ddf_commits[ddf_commits['parents'] == 'efd78a552b9bdc3147d6ac23270a8fa61dbc9844'].compute()

In [ ]:
ddf[ddf['commit_id'] == '3f5a3af4ff73c3dd3b56e1b1f008bd5562b41bcd'].compute()

In [ ]:
df_intro_commits[df_intro_commits["long_path"].isna()]['repo_id'].value_counts()

In [ ]:
ddf_commits.head()

In [ ]:
ddf_tmp = ddf_commits[ddf_commits['repo_id'].isin(outlier_projects)]

In [ ]:
ddf_tmp.head()

In [ ]:
ddf_tmp['max_file_depth_diff'].abs().describe().compute()

In [ ]:
ddf_tmp.dtypes

In [ ]:
ddf_tmp[ddf_tmp['max_file_depth_diff'].abs() > 0.0]['max_file_depth_diff'].abs().describe(percentiles=[.25, .5, .75, 0.90, 0.95, .99, .999]).compute()

In [ ]:
ddf_commits[ddf_commits['max_file_depth_diff'].abs() > 0.0]['max_file_depth_diff'].abs().describe(percentiles=[.25, .5, .75, 0.90, 0.95, .99, .999]).compute()

In [ ]:
exp_tmp_data = ddf_tmp[(ddf_tmp['max_file_depth_diff'].abs() >= 9) & (ddf_tmp['parents'].str.len() > 0)]
exp_tmp_data.compute().to_csv('exp_tmp_data_3.csv')
exp_tmp_data

In [ ]:
exp_tmp_data = ddf_commits[(ddf_commits['max_file_depth_diff'].abs() >= 11) & (ddf_tmp['parents'].str.len() > 0)]
exp_tmp_data.compute().to_csv('exp_tmp_data_4.csv')
exp_tmp_data

In [ ]:
exp_tmp_data = ddf_tmp[(ddf_tmp['max_file_depth_diff'].abs() >= 9)]
exp_tmp_data.compute().to_csv('exp_tmp_data_5.csv')
exp_tmp_data

In [ ]:
ddf_commits_with_lang = ddf_commits.merge(ddf_repoinfos[['lang']], on='repo_id')

In [ ]:
temp_data = ddf_commits_with_lang[ddf_commits_with_lang["max_file_depth_diff"].abs() > 0].compute()
temp_data["abs_diff"] = temp_data["max_file_depth_diff"].abs()
p95s_diff = temp_data.groupby('lang')["abs_diff"].describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99,0.999])

In [ ]:
p95s_diff

In [ ]:
ddf_commits_with_lang = ddf_commits_with_lang.merge(p95s_diff[["95%","99%"]], on="lang")

In [ ]:
data = ddf_commits_with_lang[ddf_commits_with_lang['max_file_depth_diff'].abs() >= ddf_commits_with_lang['99%']].compute()
data.to_csv('max_depth_diff_99_pc.csv')

In [ ]:
data_2 = data.copy()   # where the number of partitions is the number of cores you want to use

In [ ]:
# Define function to check reason
def check_reason_func(drow, mutations):
    msg = drow['message'].lower()
    if "node_modules" in msg:
        return "node_modules"
    if "vendor" in msg:
        return "vendor"
    if "init" in msg or "first" in msg:
        return "initial"

    commit_filter = (mutations['commit_id'] == drow["commit_id"]) & (mutations['repo_id'] == drow['repo_id'])
    if mutations[commit_filter]['new_path'].str.contains('node_modules/', na=False, regex=True).any() or \
            mutations[commit_filter]['old_path'].str.contains('node_modules/', na=False, regex=True).any():
        return "node_modules"
    
    if mutations[commit_filter]['new_path'].str.contains('vendor/', na=False, regex=True).any() or \
            mutations[commit_filter]['old_path'].str.contains('vendor/', na=False, regex=True).any():
        return "vendor"

    return "unknown"

In [ ]:
# Compute mutations DataFrame once
mutations = ddf[(ddf['new_path'].str.contains('node_modules/|vendor/', na=False, regex=True)) |
                (ddf['old_path'].str.contains('node_modules/|vendor/', na=False, regex=True))].compute()

In [ ]:
# Create a Dask DataFrame
ddf_data_dask = dd.from_pandas(data_2)  # Adjust the number of partitions as needed

In [ ]:
# # Define function to apply check_reason_func to each partition
# def apply_check_reason_func(partition):
#     return partition.apply(lambda row: check_reason_func(row, mutations), axis=1)

# Specify metadata for the output
meta = ('str')

# Apply function to each partition
data_3 = data_2.apply(lambda row: check_reason_func(row, mutations), axis=1, meta=meta)

In [ ]:
ddf_data_dask['output'] = result

In [ ]:
ddf_data_dask.compute().to_csv('thing_with_reason.csv')

In [ ]:
# Filtered version
thresh = 20

box_data = ddf_commits_with_lang[ddf_commits_with_lang['max_file_depth_diff'].abs() <= thresh]
box_data = box_data[box_data['max_file_depth_diff'].abs() > 0]

# outlier_data = ddf_commits_with_lang[ddf_commits_with_lang['max_file_depth_diff'] > thresh].compute()

# categories = box_data['lang'].unique()
# print (categories)

boxplot_trace = go.Box(x=box_data['lang'], y=box_data['max_file_depth_diff'].abs().compute(), name='Data')

# outlier_traces = []
# for lang in categories:
#     lang_outlier_data = outlier_data[outlier_data['lang'] == lang]
#     if not lang_outlier_data.empty:
#         trace = go.Scatter(
#             x=[lang] * len(lang_outlier_data),
#             y=lang_outlier_data['max_file_depth_diff'],
#             mode='markers+text',
#             name=lang,
#             marker=dict(color='red', size=5),
#             text=lang_outlier_data['max_file_depth_diff'],
#             textposition='middle right',
#         )
#         outlier_traces.append(trace)
#     else:
#         # Create an empty trace with category name
#         trace = go.Scatter(
#             x=[lang],
#             y=[None],  # Provide empty y-values
#             mode='markers',
#             name=lang,
#             marker=dict(color='red', size=0)  # Set size to 0 for invisible markers
#         )
#         outlier_traces.append(trace)
    
# Create the figure
fig = make_subplots(rows=2, cols=1, vertical_spacing = 0.02, row_heights=[0.25, 0.75])
fig.add_trace(boxplot_trace, row=2, col=1)

# fig.add_trace(outliers_trace, row=1, col=1)
# Add outlier scatter traces to the figure
# for trace in outlier_traces:
#     fig.add_trace(trace, row=1, col=1)

# Update layout for clarity (optional)
fig.update_layout(title='Maximum File Tree Depth Diff per Programming Language', yaxis_title='Value')

# Hide the legend
fig.update_layout(showlegend=False, height=500)

# Hide axis labels in the upper subplot
fig.update_xaxes(showticklabels=False, showgrid=False, row=1, col=1)
fig.update_yaxes(title_text='', row=1, col=1)

# Synchronize y-axis ranges between subplots
# fig.update_yaxes(range=[35, outlier_data['max_file_depth'].max()+15], row=1, col=1)
# fig.update_yaxes(range=[box_data['max_file_depth'].min()-5, 35], row=2, col=1)

# Show the plot
# fig.show()

fig.write_image('Figures/max_file_depth_diff_per_lang.pdf')

# Which levels are impaced in each commit

In [ ]:
max_depth_99_9pct = int(df_repoinfos["max_file_depth"].quantile(0.999))
max_depth_99_9pct

What is to be done:

First remove outliers.

For each commit, check which depths are affected:
- Group by commit_id
- File add -> only new file depth
- File move: old and new file depth
- File delete: old file depth

In [ ]:
ddf.columns

In [ ]:
ddf[(ddf['old_file_depth'] >= 30)].compute()

In [ ]:
commit_exclude = ['aa5154002620e307d6463eab490c66e00eee99c7', # KGN
                  '0b05c2a079225c618f3b2f3a29c1e2d5e481289a', # NAPALM
                  '63a4008a7c896d28a2277cc04c195aec746f2a2a', # packtpublishing_swift-3-functional-programming
                  # Corrections
                  '8afe8378a69a39616daba30b4eafa4b2602e17b2', # NAPALM FIX
                  '1eea640b1c86cf572e57bd83a86443c1b97a7e07' # KGN FIX
                 ]

# ddf_tmp = ddf
ddf_tmp = ddf[~ddf['commit_id'].isin(commit_exclude)]

affected_layers_per_add = ddf_tmp[(ddf_tmp['action'] == 'add')].groupby('new_file_depth').commit_id.nunique()
# affected_layers_per_move = (
#     ddf_tmp[ddf_tmp['action'] == 'move'].groupby('new_file_depth').commit_id.nunique() + 
#     ddf_tmp[ddf_tmp['action'] == 'move'].groupby('old_file_depth').commit_id.nunique() 
# )
affected_layers_per_del = ddf_tmp[(ddf_tmp['action'] == 'delete')].groupby('old_file_depth').commit_id.nunique()
affected_layers_per_ren = ddf_tmp[(ddf_tmp['action'] == 'rename')].groupby('new_file_depth').commit_id.nunique()
add_commits_per_lay, del_commits_per_lay, ren_commits_per_lay = dask.compute(affected_layers_per_add, affected_layers_per_del, affected_layers_per_ren)

# Count unique commits per 'new_file_depth' and 'old_file_depth' for 'move' actions, ensuring moves within the same depth are not double-counted
affected_layers_per_move_new = ddf_tmp[ddf_tmp['action'] == 'move'].groupby('new_file_depth').commit_id.nunique().compute()
affected_layers_per_move_old = ddf_tmp[ddf_tmp['action'] == 'move'].groupby('old_file_depth').commit_id.nunique().compute()

# Count moves within the same depth only once
moves_same_depth = ddf_tmp[(ddf_tmp['action'] == 'move') & (ddf_tmp['new_file_depth'] == ddf_tmp['old_file_depth'])].groupby('new_file_depth').commit_id.nunique().compute()

# Sum up the counts while adjusting for moves within the same depth
mov_commits_per_lay = affected_layers_per_move_new.add(affected_layers_per_move_old, fill_value=0).subtract(moves_same_depth, fill_value=0)


In [ ]:
add_commits_per_lay.sort_index()

In [ ]:
mov_commits_per_lay = affected_layers_per_move

In [ ]:
mov_commits_per_lay.sort_index()

In [ ]:
del_commits_per_lay.sort_index()

In [ ]:
combined_change_df = pd.concat([add_commits_per_lay, mov_commits_per_lay, del_commits_per_lay, ren_commits_per_lay], axis=1, keys=['Additions', 'Movements', 'Deletions', 'Renames']).fillna(0)

In [ ]:
# combined_change_df = combined_change_df.drop(122, axis=0)
combined_change_df = combined_change_df.sort_index()
combined_change_df.index = combined_change_df.index+1

In [ ]:
combined_change_df

In [ ]:
# Create the stacked bar chart figure
fig = make_subplots(cols=1,rows=4)

colors_map = {
    'Additions': px.colors.qualitative.Plotly[0], 
    'Movements': px.colors.qualitative.Plotly[2],
    'Deletions': px.colors.qualitative.Plotly[1], #'rgb(200, 25, 25)', 
    'Renames': px.colors.qualitative.Plotly[3],
}

# width = 0.85

# Add each category as a trace
for i, category in enumerate(['Additions', 'Deletions', 'Movements', 'Renames']):
    fig.add_trace(go.Bar(
        x=combined_change_df.index+1,
        y=combined_change_df[category],
        name=category,
        marker_color=colors_map[category],
        # width=width
    ), 
        row=i+1,
        col=1)

    if i == 0:
        fig.add_trace(go.Scatter(x=[58.5], y=[1], mode='markers', marker=dict(color=colors_map[category], size=10), showlegend=False), row=1,col=1)
        fig.add_annotation(x=59.5, y=0, text="1 addition<br>at level 123", showarrow=False, xshift=32.5, align='left', row=1,col=1)
    elif i == 1:
        fig.add_trace(go.Scatter(x=[58.5], y=[1], mode='markers', marker=dict(color=colors_map[category], size=10), showlegend=False), row=2,col=1)
        fig.add_annotation(x=59.5, y=0, text="1 deletion<br>at level 123", showarrow=False, xshift=32.5, align='left', row=2,col=1)
    fig.update_xaxes(title='Layer', row=i+1, col=1)
    fig.update_yaxes(title='Number of Commits (log)', row=i+1, col=1)


# Set the barmode to 'stack'
fig.update_layout(
    height=1000,
    width=1000,
    margin=dict(t=20,r=20,b=20,l=20),
    template='ggplot2',
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 5
    )
)

fig.update_xaxes(range=[0.5,59.5])
fig.update_yaxes(type="log")

# Show the plot
fig.show()
fig.write_image(f"Figures/bar_all_file_stuc_per_lay.pdf")


In [ ]:
# Create the stacked bar chart figure
fig = go.Figure()

colors_map = {
    'Additions': px.colors.qualitative.Plotly[0], 
    'Movements': px.colors.qualitative.Plotly[2],
    'Deletions': px.colors.qualitative.Plotly[1], #'rgb(200, 25, 25)', 
    'Renames': px.colors.qualitative.Plotly[3],
}

# width = 0.85

# Add each category as a trace
for i, category in enumerate(['Additions', 'Deletions', 'Movements', 'Renames']):
    fig.add_trace(go.Bar(
        x=combined_change_df.index,
        y=combined_change_df[category],
        name=category,
        marker_color=colors_map[category],
        # width=width
    ))
    
fig.update_xaxes(title='Layer')
fig.update_yaxes(title='Number of Commits (log)')


# Set the barmode to 'stack'
fig.update_layout(
    barmode='group',
    height=400,
    width=1000,
    margin=dict(t=20,r=20,b=20,l=20),
    template='ggplot2',
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 5
    )
)

fig.update_xaxes(range=[0.5,33.5])
fig.update_yaxes(type="log")

# Show the plot
fig.show()
fig.write_image("Figures/bar_all_file_stuc_per_lay.pdf")


In [ ]:
ddf[(ddf['action'] == 'add') & (ddf['new_file_depth'] > 33)].compute()

In [ ]:
# combined_change_df.index.to_list().describe()

In [ ]:
aaffected_layers_per_add = ddf[(ddf['action'] == 'add')]['new_file_depth']
aaffected_layers_per_move = ddf[ddf['action'] == 'move']['new_file_depth'] + ddf[ddf['action'] == 'move']['old_file_depth']
aaffected_layers_per_del = ddf[(ddf['action'] == 'delete')]['old_file_depth']

In [ ]:
# fig = px.box(aaffected_layers_per_add)

In [ ]:
# fig.show()

In [ ]:
aaffected_layers_per_add.describe().compute()

In [ ]:
# ddf[(ddf['action'] == 'delete') & (ddf["old_file_depth"] >= 25)].compute()

In [ ]:
# Plot additions
fig = go.Figure(data=[go.Bar(x=combined_change_df.index, y=combined_change_df['Additions'])])
fig.update_layout(title='Additions by Layer', xaxis_title='Layers', yaxis_title='Number of Commits')
fig.show()

# Plot movements
fig = go.Figure(data=[go.Bar(x=combined_change_df.index, y=combined_change_df['Movements'])])
fig.update_layout(title='Movements by Layer', xaxis_title='Layers', yaxis_title='Number of Commits')
fig.show()

# Plot deletions
fig = go.Figure(data=[go.Bar(x=combined_change_df.index, y=combined_change_df['Deletions'])])
fig.update_layout(title='Deletions by Layer', xaxis_title='Layers', yaxis_title='Number of Commits')
fig.show()


In [ ]:
affected_layers_per_move = ddf[ddf['action'] == 'move'][['old_file_depth', 'new_file_depth']]
affected_layer_combinations = affected_layers_per_move.groupby(['old_file_depth', 'new_file_depth']).size().compute().reset_index(name='counts')

In [ ]:
affected_layer_combinations

In [ ]:


# Create the heatmap figure
fig = go.Figure(data=go.Heatmap(
    z=affected_layer_combinations['counts'],
    x=affected_layer_combinations['old_file_depth'],
    y=affected_layer_combinations['new_file_depth'],
    colorscale='sunsetdark',
    reversescale=True,
    hoverongaps=False))

# Set the axis labels
fig.update_layout(
    title="File Depth Correlation for Moved Files",
    xaxis_title='Old File Depth',
    yaxis_title='New File Depth')

# Make the figure a bit higher
fig.update_layout(
    height=800
)

# Show the heatmap
fig.show()
fig.write_image('Figures/move_file_depth_correlation.pdf')


In [ ]:
# Create the heatmap figure
fig = go.Figure(data=go.Heatmap(
    z=np.log10(affected_layer_combinations['counts']),
    x=affected_layer_combinations['old_file_depth'],
    y=affected_layer_combinations['new_file_depth'],
    colorscale='sunsetdark',
    reversescale=True,
    hoverongaps=False))

# Set the axis labels
fig.update_layout(
    xaxis_title='Old File Depth',
    yaxis_title='New File Depth')

# Make the figure a bit higher
fig.update_layout(
    height=800
)

# Show the heatmap
fig.show()


In [ ]:
affected_layer_combinations['file_depth_diff'] = (affected_layer_combinations['new_file_depth']-affected_layer_combinations['old_file_depth']).abs()

In [ ]:
affected_layer_combinations.sort_values('file_depth_diff', ascending=False)

In [ ]:
count_diff = affected_layer_combinations.groupby('file_depth_diff')['counts'].count()

In [ ]:
fig = px.bar(count_diff)
fig.show()

In [ ]:
outlier_layers = affected_layer_combinations[affected_layer_combinations['file_depth_diff'] >= 15]
outlier_layers

In [ ]:
indexes = list(zip(outlier_layers.old_file_depth, outlier_layers.new_file_depth))
indexes

In [ ]:
ddf['file_depth_diff'] = (ddf['new_file_depth']-ddf['old_file_depth']).abs()

In [ ]:
ddf[(ddf['file_depth_diff'] >= 13) & (ddf['action'] == 'move')].compute()

In [ ]:
file_depth_diff_occurences = ddf[ddf['action'] == 'move'].groupby('file_depth_diff').count()['commit_id'].compute().sort_index()

In [ ]:
file_depth_diff_occurences

In [ ]:
file_depth_diff_occurences = file_depth_diff_occurences.to_frame().style.format(decimal=',', thousands='.', precision=2)

In [ ]:
print(file_depth_diff_occurences.to_latex())

In [ ]:
ddf[(ddf['file_depth_diff'] >= 12) & (ddf['action'] == 'move')].compute().to_excel('large_moves.xlsx')

# See how many layers are affected per commit

In [ ]:
# Function to combine and count unique depths
def count_unique_depths(group):
    combined_depths = pd.concat([group['new_file_depth'], group['old_file_depth']])
    unique_depths = combined_depths[combined_depths >= 0].unique()
    return len(unique_depths)
    
# Function to combine and count unique depths
def partition_func(partition):
    group = partition.groupby(['repo_id', 'commit_id'])
    return group.apply(count_unique_depths, include_groups=False).reset_index()

res = ddf.map_partitions(partition_func, meta={'repo_id': 'str', 'commit_id': 'str', 0: 'i8'})
# result = ddf.groupby(['repo_id', 'commit_id']).apply(count_unique_depths, meta=('x', 'i8')).reset_index().compute(scheduler='threads')
res.compute(scheduler='threads')

In [ ]:
# Melt the DataFrame to stack new_file_depth and old_file_depth into a single column
melted_ddf = dd.melt(ddf, id_vars=['repo_id', 'commit_id'], value_vars=['new_file_depth', 'old_file_depth'], value_name='file_depth')

# Group by repo_id and commit_id, then calculate unique count of file_depth
result = melted_ddf[melted_ddf['file_depth'] >= 0].groupby(['repo_id', 'commit_id'])['file_depth'].nunique().reset_index().compute()


In [ ]:
result.describe()

In [ ]:
result

In [ ]:
result.to_parquet('affected_layers.parquet', compression='brotli')

In [ ]:
result = pd.read_parquet('affected_layers.parquet')

In [ ]:
result['file_depth'].describe(percentiles=[0.25,0.5,0.75,0.9,0.99,0.999])

In [ ]:
grouped_repo_affeclay = result.groupby('repo_id').describe(percentiles=[0.25,0.5,0.75,0.9,0.99,0.999])

In [ ]:
grouped_repo_affeclay

In [ ]:
grouped_repo_affeclay.columns

In [ ]:
fig = px.scatter(x=grouped_repo_affeclay[('file_depth', 'count')], y=grouped_repo_affeclay[('file_depth',   'mean')])
fig.show()

In [ ]:
grouped_repo_affeclay.mode()

In [ ]:
no_affected_layer_per_commit = result.groupby('file_depth').count()

In [ ]:
no_affected_layer_per_commit['commit_id']/no_affected_layer_per_commit['commit_id'].sum()*100

In [ ]:
no_affected_layer_per_commit = no_affected_layer_per_commit.drop([52, 57], axis=0)

In [ ]:
fig = px.bar(no_affected_layer_per_commit[['commit_id']])
fig.update_yaxes(type='log', title='Number of Commits')
fig.update_xaxes(title='Number of affected layers')
fig.update_layout(showlegend=False, width=1000, height=400, template='ggplot2')
fig.update_traces(marker_color='rgb(200, 25, 25)')

# fig.show()
fig.write_image('Figures/bar_no_affected_layers.pdf')
fig.show()

In [ ]:
result[result['file_depth'] >= 25]

# Affected layers per (linguist) file type

In [ ]:
# Create the stacked bar chart figure
fig = go.Figure()

# Add each category as a trace
for category in linguist_types:
    consider_langs = [a for a in linguist_langs if linguist_langs[a]['type'] == category]
    # affected_layers_per_add = ddf[(ddf['action'] == 'add') & ddf['language'].isin(consider_langs)].groupby('new_file_depth').commit_id.nunique()
    # affected_layers_per_move = ddf[(ddf['action'] == 'move') & ddf['language'].isin(consider_langs)].groupby('new_file_depth').commit_id.nunique() + ddf[(ddf['action'] == 'move')& ddf['language'].isin(consider_langs)].groupby('new_file_depth').commit_id.nunique()
    # affected_layers_per_del = ddf[((ddf['action'] == 'delete') & ddf['language'].isin(consider_langs))].groupby('new_file_depth').commit_id.nunique()
    data = ddf[ddf['language'].isin(consider_langs)].groupby('new_file_depth').commit_id.nunique()
    fig.add_trace(go.Bar(
        x=data.index,
        y=data,
        name=category
    ))

# Set the barmode to 'stack'
fig.update_layout(
    barmode='stack',
    title='File Structure Changes by Layer',
    xaxis_title='Layers',
    yaxis_title='Number of Commits'
)

# # Show the plot
fig.show()
fig.write_image("Figures/changes_per_type_per_layer.pdf")

# Insight between the individual and corporation projects

In [ ]:
individual_projects = df_github[df_github['owner.type'] == "User"]
organisation_projects = df_github[df_github['owner.type'] == "Organization"]

In [ ]:
ddf_repoinfos.compute()

In [ ]:
df_individual_project_repoinfos = ddf_repoinfos[ddf_repoinfos.index.isin(individual_projects['repo_id'])].compute()
df_organisation_project_repoinfos = ddf_repoinfos[ddf_repoinfos.index.isin(organisation_projects['repo_id'])].compute()

In [ ]:
df_organisation_project_repoinfos

In [ ]:
df_individual_project_repoinfos

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(y=df_organisation_project_repoinfos["no_commits"], name='Organization'))
fig.add_trace(go.Box(y=df_individual_project_repoinfos["no_commits"], name='User'))

fig.update_layout(title='No Commits Organization vs User',
                  xaxis_title='User Type',
                  yaxis_title='Number of Commits')

fig.show()

In [ ]:
df_organisation_project_repoinfos["no_commits"].describe(percentiles=[.25, .5, .75, .99, .999]), df_individual_project_repoinfos["no_commits"].describe(percentiles=[.25, .5, .75, .99, .999])

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(y=df_organisation_project_repoinfos["no_contributors"], name='Organization'))
fig.add_trace(go.Box(y=df_individual_project_repoinfos["no_contributors"], name='User'))

fig.update_layout(title='No Contributors of User vs Organization',
                  xaxis_title='User Type',
                  yaxis_title='Number of Contributors')

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Box(y=df_organisation_project_repoinfos["max_file_depth"], name='Organization'))
fig.add_trace(go.Box(y=df_individual_project_repoinfos["max_file_depth"], name='User'))

fig.update_layout(title='Max Depth of User vs Organization',
                  xaxis_title='Project Type',
                  yaxis_title='Max File Depth')

fig.show()

In [ ]:
df_organisation_project_repoinfos["max_file_depth"].describe(percentiles=[.25, .5, .75, .99, .999]), df_individual_project_repoinfos["max_file_depth"].describe(percentiles=[.25, .5, .75, .99, .999])

In [ ]:
%%time 
ddf_commits.compute()

In [ ]:
%%time 
ddf_commits['commit_time'].compute()

In [ ]:
# FIX FOR STATS PER COMMIT

# df_commits_temp = ddf_commits.compute()

# ddf_adds_per_commit = ddf[ddf["action"] == "add"].groupby("commit_id").size()
# ddf_dels_per_commit = ddf[ddf["action"] == "delete"].groupby("commit_id").size()
# ddf_moves_per_commt = ddf[ddf["action"] == "move"].groupby("commit_id").size()
# ddf_rename_per_commt = ddf[ddf["action"] == "rename"].groupby("commit_id").size()

# adds_per_commit, dels_per_commit, moves_per_commt, rename_per_commt = dd.compute(ddf_adds_per_commit, ddf_dels_per_commit, ddf_moves_per_commt, ddf_rename_per_commt)

# df_commits_temp["n_added"] = (
#     adds_per_commit
# )

# df_commits_temp["n_deleted"] = (
#     dels_per_commit
# )

# df_commits_temp["n_moved"] = (
#     moves_per_commt
# )

# df_commits_temp["n_renamed"] = (
#     rename_per_commt
# )

# df_commits_temp = df_commits_temp.fillna({"n_added": 0, "n_deleted": 0, "n_moved": 0, "n_renamed": 0})

# df_commits_temp["n_added"] = df_commits_temp["n_added"].astype(np.int32)
# df_commits_temp["n_deleted"] = df_commits_temp["n_deleted"].astype(np.int32)
# df_commits_temp["n_moved"] = df_commits_temp["n_moved"].astype(np.int32)
# df_commits_temp["n_renamed"] = df_commits_temp["n_renamed"].astype(np.int32)
# df_commits_temp["n_struc_changes"] = (
#     df_commits_temp["n_added"]
#     + df_commits_temp["n_deleted"]
#     + df_commits_temp["n_moved"]
#     + df_commits_temp["n_renamed"]
# )

# df_commits_temp["max_file_depth"] = df_commits_temp["total_files_per_level"].apply(
#     lambda lst: len(lst.split(","))
#     - len(list(itertools.takewhile(lambda x: x != "0", reversed(lst.split(","))))),
# )

# df_commits_temp.to_parquet(
#     f"../hpc-datasets/commits-idx-fixed.parquet",
#     compression="brotli",
#     partition_cols="bucket",
# )


In [ ]:
(ddf_repoinfos['no_struc_changing_commits']/ddf_repoinfos['no_commits']).compute().mean()

# Average num changes per commit

In [ ]:
ddf_commits[['n_added',	'n_deleted',	'n_moved',	'n_renamed']].describe().compute()

# Check how projects grow (horiz/vert)

In [ ]:
num_commits_per_proj = ddf_commits.groupby('repo_id').count().compute()["commit_time"]

In [ ]:
single_commit_repos = list(num_commits_per_proj[num_commits_per_proj <= 1].index)

In [ ]:
# ddf_file_additions = ddf_commits[ddf_commits["action"] == "add"]

# For each repository, calculate the percentage of commits that have increased the max file depth
def has_max_depth_increased(df):
    df = df.sort_values('commit_index')
    return (df["max_file_depth"] > df["max_file_depth"].shift(1))[1:].sum() / (len(df)-1)

perc_commits_max_depth_increased = ddf_commits[~ddf_commits["repo_id"].isin(single_commit_repos)].groupby("repo_id").apply(has_max_depth_increased, meta=("x", "float64"))

In [ ]:
tmp_commits = ddf_commits[~ddf_commits["repo_id"].isin(single_commit_repos) & (ddf_commits['parents'] != '')]
tmp_commits['depth_inc'] = tmp_commits['max_file_depth_diff'] > 0
tmp_commits['depth_same'] = tmp_commits['max_file_depth_diff'] == 0
tmp_commits['depth_dec'] = tmp_commits['max_file_depth_diff'] < 0

describe_inc = tmp_commits.groupby("repo_id")['depth_inc'].mean().describe()
describe_same = tmp_commits.groupby("repo_id")['depth_same'].mean().describe()
describe_dec = tmp_commits.groupby("repo_id")['depth_dec'].mean().describe()

ddescribe_inc, ddescribe_same, ddescribe_dec = dd.compute(describe_inc, describe_same, describe_dec)
ddescribe_inc, ddescribe_same, ddescribe_dec

In [ ]:
tmp_commits[['depth_inc', 'depth_same', 'depth_dec']].mean().compute()

In [ ]:
(
    tmp_commits[tmp_commits['depth_dec']]
    .merge(df_repoinfos[['lang']], on='repo_id')
    .compute()
    .groupby('lang')['max_file_depth_diff'].describe(percentiles=[0.001, 0.01, 0.1, 0.25, 0.75])
)

In [ ]:
(
    tmp_commits[tmp_commits['depth_inc']]
    .merge(df_repoinfos[['lang']], on='repo_id')
    .compute()
    .groupby('lang')['max_file_depth_diff'].describe(percentiles=[0.25,0.75,0.90,0.99,0.999,0.9999])
)

In [ ]:
tmp_commits[tmp_commits['depth_dec']]['max_file_depth_diff'].describe(percentiles=[0.001,0.01, 0.1, 0.25, 0.75]).compute()

In [ ]:
tmp_commits[tmp_commits['depth_inc']]['max_file_depth_diff'].describe(percentiles=[0.25,0.75,0.90,0.99,0.999,0.9999]).compute()

In [ ]:
tmp_commits[(tmp_commits['max_file_depth_diff'] >= 10.96)].compute()

In [ ]:
tmp_commits[tmp_commits['max_file_depth_diff'] <= -8.360000].compute()

In [ ]:
tmp_commits[tmp_commits['max_file_depth_diff'] <= -7].compute()

In [ ]:
tmp_commits.shape[0].compute()

In [ ]:
ddf_commits.shape[0].compute()

In [ ]:
ddf[['repo_id', 'commit_id']].drop_duplicates().shape[0].compute()

In [ ]:
# For each repository, calculate the percentage of commits that have increased the max file depth
def has_max_depth_decreased(df):
    df = df.sort_values('commit_index')
    return (df["max_file_depth"] < df["max_file_depth"].shift(1))[1:].sum() / (len(df)-1)

perc_commits_max_depth_decreased = ddf_commits[~ddf_commits["repo_id"].isin(single_commit_repos)].groupby("repo_id").apply(has_max_depth_decreased, meta=("x", "float64"))

In [ ]:
# For each repository, calculate the percentage of commits that have increased the max file depth
def has_max_depth_same(df):
    df = df.sort_values('commit_index')
    return (df["max_file_depth"] == df["max_file_depth"].shift(1))[1:].sum() / (len(df)-1)

perc_max_depth_stayed_same = ddf_commits[~ddf_commits["repo_id"].isin(single_commit_repos)].groupby("repo_id").apply(has_max_depth_same, meta=("x", "float64"))

In [ ]:
perc_depth_inc, perc_depth_dec, perc_depth_same = dd.compute(perc_commits_max_depth_increased, perc_commits_max_depth_decreased, perc_max_depth_stayed_same)

In [ ]:
perc_depth_inc.describe(), perc_depth_dec.describe(), perc_depth_same.describe()

In [ ]:
fig = px.histogram(perc_depth_inc, nbins=100)
fig.show()

In [ ]:
fig = px.histogram(perc_depth_same, nbins=30)
fig.show()

In [ ]:
fig = px.histogram(perc_depth_dec, nbins=30)
fig.show()

In [ ]:
perc_depth_dec[perc_depth_dec==0].count() / len(perc_depth_dec)

In [ ]:
perc_depth_inc[perc_depth_inc==0].count() / len(perc_depth_inc)

In [ ]:
perc_depth_same[perc_depth_same==0]

In [ ]:
perc_depth_same[perc_depth_same==0].count() / len(perc_depth_same)

In [ ]:
perc_depth_dec[perc_depth_dec>0].describe()

In [ ]:
perc_depth_inc[perc_depth_inc>0].describe()

In [ ]:
perc_depth_same[perc_depth_same>0].describe()

In [ ]:
perc_depth_same[perc_depth_same==0]

In [ ]:
bla = ddf_commits[ddf_commits["repo_id"] == "auspicious3000_autovc"].compute()

In [ ]:
(bla["max_file_depth"] > bla["max_file_depth"].shift(1))[1:].sum()

In [ ]:
test_df = ddf_commits[ddf_commits["repo_id"] == "auspicious3000_autovc"].compute()

In [ ]:
test_df.head()

In [ ]:
data = (test_df["max_file_depth"] > test_df["max_file_depth"].shift(1))[1:]
data2 = (test_df["max_file_depth"] < test_df["max_file_depth"].shift(1))[1:]
data3 = (test_df["max_file_depth"] == test_df["max_file_depth"].shift(1))[1:]

In [ ]:
data.sum(), len(test_df)

In [ ]:
data.sum() / len(test_df)

In [ ]:
data2.sum() / len(test_df)

In [ ]:
data3.sum() / len(test_df)

In [ ]:
tmp_df = ddf_commits.groupby('repo_id').count().compute()["commit_time"]

In [ ]:
(tmp_df == 1).sum()

In [ ]:
(tmp_df > 1).sum()

In [ ]:
perc_depth_dec[perc_depth_dec==1]

# Stability score 

In [ ]:
ddf_commits_per_repo = ddf_commits.groupby('repo_id')
ddf_all_commits_per_repo = ddf_commits.groupby('repo_id')

In [ ]:
subddf = ddf_commits[['n_added', 'n_deleted', 'n_moved', 'n_renamed', 'n_struc_changes', 'max_file_depth']]

In [ ]:
ddf_all_commits = ddf_all_commits.set_index('commit_id')

In [ ]:
abc1 = ddf_all_commits.reset_index()
abc2 = subddf.reset_index()
merged_ddf = abc1.merge(abc2,how="left")
merged_ddf = merged_ddf.fillna(0).astype({'n_added': 'int32', 'n_deleted': 'int32', 'n_moved': 'int32', 'n_renamed': 'int32', 'n_struc_changes': 'int32', 'max_file_depth': 'int16'})

In [ ]:
proj = merged_ddf[merged_ddf["repo_id"] == "vapor_vapor"].compute()

In [ ]:
proj

In [ ]:
from data_analysis.stability import compute_depth_stability

In [ ]:
ent_data = merged_ddf.groupby('repo_id').apply(lambda r: compute_depth_stability(r), meta=('x', 'f8')).compute()

In [ ]:
ent_data.describe()

In [ ]:
ent_data_2 = merged_ddf.groupby('repo_id').apply(lambda r: compute_depth_stability(r, time_interval="daily"), meta=('x', 'f8'))

In [ ]:
ent_data_2.describe().compute()

In [ ]:
import gc
gc.collect()

In [ ]:
merged_ddf.dtypes

In [ ]:
ddf_all_commits

In [ ]:
ddf_commits

In [ ]:
df_all_commits = ddf_all_commits.compute()

In [ ]:
df_all_commits = df_all_commits.set_index(['repo_id','commit_id'])

In [ ]:
ddf_commits

In [ ]:
df_commits_changes = ddf_commits[['repo_id','commit_id','n_struc_changes','max_file_depth_diff']].compute().set_index(['repo_id','commit_id'])


In [ ]:
df_merged = df_all_commits.merge(df_commits_changes, how='left',left_index=True, right_index=True).fillna({'n_struc_changes': 0, 'max_file_depth_diff': 0 })

In [ ]:
df_merged.reset_index().to_parquet('../hpc-datasets/commits_merged.parquet', partition_cols=['bucket'], compression='brotli')

In [ ]:
ddf_merged = dd.read_parquet('../hpc-datasets/commits_merged.parquet')

In [ ]:
# Convert commit dates to datetime
ddf_merged['commit_time'] = dd.to_datetime(ddf_merged['commit_time'], unit='s')

In [ ]:
ddf_merged

In [ ]:
# Add absolute depth diff
ddf_merged['abs_max_file_depth_diff'] = ddf_merged['max_file_depth_diff'].abs()

In [ ]:
# Group by project and calculate the first commit date for each project
project_first_commit_date = ddf_all_commits.groupby('repo_id')['commit_time'].min().compute()
project_first_commit_date = pd.to_datetime(project_first_commit_date, unit="s")

# Calculate the age of each project based on the first commit date
ddf_merged['project_age'] = ddf_merged['repo_id'].map(project_first_commit_date)
ddf_merged['project_age'] = (ddf_merged['commit_time'] - ddf_merged['project_age']).dt.days

In [ ]:
# Group by project and calculate the first commit date for each project
project_last_commit_date = ddf_all_commits.groupby('repo_id')['commit_time'].max().compute()
project_last_commit_date = pd.to_datetime(project_last_commit_date, unit="s")

df_repoinfos['project_age'] = (project_last_commit_date-project_first_commit_date).dt.days

In [ ]:
ddf_merged.dtypes

In [ ]:
ddf_merged.head()

In [ ]:
ddf_merged[ddf_merged['repo_id'] == 'codebude_qrcoder'].compute()

In [ ]:
# Extract the week of the year for resampling
ddf_merged['week'] = ddf_merged['commit_time'].dt.to_period('W')

# Group by project and week, then aggregate
weekly_data = ddf_merged.groupby(['repo_id', 'project_age', 'week']).agg({
    'commit_id': 'count',
    'n_struc_changes': 'sum',
    'abs_max_file_depth_diff': 'sum' 
}).rename(columns={'commit_id': 'total_commits'}).reset_index()

# Calculate structural change rate
weekly_data['change_rate'] = weekly_data['n_struc_changes'] / weekly_data['total_commits']

# Handle cases with no commits to avoid division by zero
weekly_data['change_rate'] = weekly_data['change_rate'].fillna(0)

# Calculate the inverse change rate as a stability index
weekly_data['stability_index'] = 1 / (1 + weekly_data['change_rate'] + weekly_data['abs_max_file_depth_diff'])

In [ ]:
weekly_data[weekly_data['repo_id'] == "vapor_vapor"].compute()

In [ ]:
# Normalize the stability index within each project
def normalize_column(df):
    col_min = df.min()
    col_max = df.max()
    return (df - col_min) / (col_max - col_min)

weekly_data['normalized_stability'] = weekly_data.groupby('repo_id')['stability_index'].transform(normalize_column)

In [ ]:
# Function to calculate stability scores for different project percentiles
def calculate_stability_at_percentiles(weekly_data, percentile):
    # Find the age percentile threshold for each project
    project_age_percentiles = np.round(df_repoinfos["project_age"]*(percentile/100)).to_frame("age_threshold")

    # Merge with weekly_data to filter projects based on their age
    filtered_weekly_data = dd.merge(weekly_data, project_age_percentiles, left_on='repo_id', right_index=True, how='inner')
    filtered_weekly_data = filtered_weekly_data[filtered_weekly_data['project_age'] <= filtered_weekly_data['age_threshold']]

    # Aggregate to get a single stability score for the project segment
    stability_scores = filtered_weekly_data.groupby('repo_id')['normalized_stability'].mean()
    
    # Convert to DataFrame for further processing
    stability_scores_df = stability_scores.reset_index().rename(columns={'normalized_stability': f'stability_score_{percentile}'})
    
    return stability_scores_df

In [ ]:
# Calculate stability scores at specified percentiles
percentiles = [10, 25, 37.5, 50, 62.5, 75, 90, 100]
stability_scores = dd.compute([calculate_stability_at_percentiles(weekly_data, percentile) for percentile in percentiles], scheduler='threads')

# Merge stability scores for different percentiles
stability_scores_merged = pd.concat([score_df.set_index('repo_id') for score_df in stability_scores[0]], axis=1)

In [ ]:
for percentile in percentiles:
    # Normalize stability scores across all projects for comparison
    stability_min = stability_scores_merged[f'stability_score_{percentile}'].min()
    stability_max = stability_scores_merged[f'stability_score_{percentile}'].max()
    
    if stability_max - stability_min != 0:
        stability_scores_merged[f'stability_score_{percentile}'] = (stability_scores_merged[f'stability_score_{percentile}'] - stability_min) / (stability_max - stability_min)
    else:
        stability_scores_merged[f'stability_score_{percentile}'] = 1

In [ ]:
stability_scores_merged.describe()

In [ ]:
stability_scores_merged.loc["10se1ucgo_disablewintracking"]

In [ ]:
tmp_data = ddf_merged[ddf_merged['repo_id'] == "10se1ucgo_disablewintracking"].compute()

In [ ]:
fig = px.scatter(tmp_data, x='project_age', y='n_struc_changes')
for percentile in percentiles:
    age_tresh = np.round(df_repoinfos["project_age"].loc["10se1ucgo_disablewintracking"]*(percentile/100))
    fig.add_vline(age_tresh)
fig.show()

In [ ]:
stability_scores_merged.describe(percentiles=[0.1,0.25,0.75,0.9])

In [ ]:
stability_scores_merged[stability_scores_merged['stability_score_100'] >= stability_scores_merged['stability_score_100'].quantile(0.99)]

In [ ]:
fig = px.violin(stability_scores_merged[['stability_score_10','stability_score_25','stability_score_50','stability_score_75','stability_score_90','stability_score_100']].rename(columns={
                     "stability_score_10": "10%",
                     "stability_score_25": "25%",
                     "stability_score_50": "50%",
                     "stability_score_75": "75%",
                     "stability_score_90": "90%",
                     "stability_score_100": "100%"
                 }), 
                box=True, # draw box plot inside the violin
                points='outliers', # can be 'outliers', or False
               )
fig.update_layout(template='ggplot2')
fig.update_traces(marker_color='rgb(200, 25, 25)', meanline_visible=True)
fig.update_yaxes(title="Stability Score")
fig.update_xaxes(title="Project Lifespan")
fig.show()
fig.update_layout(width=1000, height=500)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.write_image("Figures/violin_stability_5_points.pdf")

In [ ]:
stability_scores_merged['stability_score_100'].quantile(0.99)

In [ ]:

fig = px.violin(stability_scores_merged['stability_score_100'].rename("100%"), 
                box=True, # draw box plot inside the violin
                points='outliers', # can be 'outliers', or False
               )
fig.update_layout(template='ggplot2')
fig.update_traces(marker_color='rgb(200, 25, 25)', meanline_visible=True)
fig.update_yaxes(title="Stability Score")
fig.update_xaxes(title="Project Lifespan")
fig.show()
fig.update_layout(width=1000, height=500)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.write_image("Figures/violin_stability_full_history.pdf")

In [ ]:
fig = go.Figure()
fig_data = (
    stability_scores_merged.rename(columns={
         "stability_score_10": "10%",
         "stability_score_37.5": "37.5%",
         "stability_score_25": "25%",
         "stability_score_50": "50%",
         "stability_score_62.5": "62.5%",
         "stability_score_75": "75%",
         "stability_score_90": "90%",
         "stability_score_100": "100%"
     })
    .describe()
    .loc[['25%','50%','75%', 'mean']]
    .T
    .rename(columns={'25%': '25th percentile','50%': 'median','75%': '75th percentile'})
)

names = ['25th percentile', 'median', '75th percentile', 'mean']
color = ['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                       'rgb(44, 160, 44)', 'rgb(200, 25, 25)',]

for i, col in enumerate(fig_data.columns):
    fig.add_trace(
        go.Scatter(
            x=fig_data.index,
            y=fig_data[col],
            line=dict(color=color[i], width=1, dash='dot'),
            name=names[i]
        )
    )
fig.update_layout(template='ggplot2')
# fig.update_traces(marker_color='rgb(200, 25, 25)')
fig.update_yaxes(title="Stability Score")
fig.update_xaxes(title="Project Lifespan")
fig.update_layout(width=1000, height=300)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.show()
fig.write_image("Figures/line_stability_describe.pdf")

In [ ]:
stability_scores_merged_repoinfo = stability_scores_merged.merge(df_repoinfos, on='repo_id')

In [ ]:
fig = px.scatter(stability_scores_merged_repoinfo, x='no_commits', y='stability_score_100')
fig.show()

In [ ]:
fig = px.scatter(stability_scores_merged_repoinfo, x='no_contributors', y='stability_score_100', log_x=True)
fig.show()

In [ ]:
type_counts = df_repoinfos.groupby('owner.type').size()
type_counts

In [ ]:
fig = go.Figure()
tmp_proj = stability_scores_merged_repoinfo[stability_scores_merged_repoinfo['owner.type'] == "User"]['stability_score_100']
fig.add_trace(go.Violin(x=['User<br>(56243 projects)'] * len(tmp_proj), y=tmp_proj))
              
tmp_proj = stability_scores_merged_repoinfo[stability_scores_merged_repoinfo['owner.type'] == "Organization"]['stability_score_100']
fig.add_trace(go.Violin(x=['Organization<br>(38004 projects)'] * len(tmp_proj), y=tmp_proj))

fig.update_layout(template='ggplot2', showlegend=False)
fig.update_traces(marker_color='rgb(200, 25, 25)', meanline_visible=True, box_visible=True)
fig.update_yaxes(title="Stability Score")
fig.update_xaxes(title="Project Lifespan")
fig.show()
fig.update_layout(width=1000, height=500)
fig.write_image("Figures/violin_stability_owner_type.pdf")

In [ ]:
stability_scores_merged_repoinfo.groupby('owner.type')['stability_score_100'].describe()

In [ ]:
df_repoinfos['struc_chnging_perc'] = (ddf_commits.groupby('repo_id').count().compute()/ddf_all_commits.groupby('repo_id').count().compute())['commit_id']

In [ ]:
fig = px.scatter(stability_scores_merged.merge(df_repoinfos, on='repo_id'), x='struc_chnging_perc', y='stability_score_100')
fig.show()

In [ ]:
fig = px.scatter(ddf_merged[ddf_merged['repo_id'] == "alipay_alipay-sdk-php-all"].compute(), x='commit_time', y='n_struc_changes')
fig.show()

In [ ]:
fig = px.scatter(ddf_merged[ddf_merged['repo_id'] == "torvalds_linux"].compute(), x='commit_time', y='n_struc_changes')
fig.show()

In [ ]:
# Group by project and calculate the first commit date for each project
project_first_commit_date = ddf_merged.groupby('repo_id')['commit_time'].min().compute()


In [ ]:
# Calculate the age of each project based on the first commit date
ddf_merged['project_age'] = ddf_merged['repo_id'].map(project_first_commit_date)
ddf_merged['project_age'] = (ddf_merged['commit_time'] - ddf_merged['project_age']).dt.days


In [ ]:
# Function to calculate stability scores for different project percentiles
def calculate_stability_at_percentiles(weekly_data, percentile):
    # Find the age percentile threshold for each project
    project_age_percentiles = weekly_data.groupby('repo_id')['project_age'].apply(lambda x: x.quantile(percentile / 100)).to_frame('age_threshold')

    # Merge with weekly_data to filter projects based on their age
    filtered_weekly_data = dd.merge(weekly_data, project_age_percentiles, left_on='repo_id', right_index=True, how='inner')
    filtered_weekly_data = filtered_weekly_data[filtered_weekly_data['project_age'] <= filtered_weekly_data['age_threshold']]

    # Apply the function to the filtered data and compute the results
    stability_scores = filtered_weekly_data.groupby('repo_id').apply(calculate_normalized_stability).compute()

    # Convert to DataFrame for further processing
    stability_scores_df = stability_scores.reset_index().rename(columns={0: f'stability_score_{percentile}'})
    
    return stability_scores_df
    
# Calculate stability scores at 25th, 50th, and 75th percentiles
stability_scores_25 = calculate_stability_at_percentiles(weekly_data, 25)
stability_scores_50 = calculate_stability_at_percentiles(weekly_data, 50)
stability_scores_75 = calculate_stability_at_percentiles(weekly_data, 75)


In [ ]:
stability_scores_10 = calculate_stability_at_percentiles(weekly_data, 10)


In [ ]:

# stability_scores_df = stability_scores_df.set_index('repo_id')
stability_scores_df['stability_score_10'] = stability_scores_10.set_index('repo_id')['stability_score_10']
stability_scores_df['stability_score_25'] = stability_scores_25.set_index('repo_id')['stability_score_25']
stability_scores_df['stability_score_50'] = stability_scores_50.set_index('repo_id')['stability_score_50']
stability_scores_df['stability_score_75'] = stability_scores_75.set_index('repo_id')['stability_score_75']

In [ ]:
stability_scores_df = stability_scores_df.rename(columns={'normalized_stability_score': 'stability_score_100'})

In [ ]:
stability_scores_df.loc["vapor_vapor"]

In [ ]:
stability_scores_df.loc["torvalds_linux"]

In [ ]:
stability_scores_df.describe()

In [ ]:
px.violin(stability_scores_df, violinmode='overlay')

In [ ]:
px.violin(stability_scores_df.merge(df_repoinfos, on='repo_id'), violinmode='overlay', x='owner.type', y='stability_score_100')

In [ ]:
stability_scores_df.merge(df_repoinfos, on='repo_id').groupby('owner.type')['stability_score_100'].describe()

In [ ]:
ddf_merged

In [ ]:
fig = px.scatter(x=ddf_merged['project_age'].compute(), y=stability_scores_df['stability_score_100'])
fig.show()

In [ ]:
ddf_merged['project_age'].value_counts().compute()

In [ ]:
ddf_merged[ddf_merged['repo_id'] == "jordansinger_uidraw"].compute()

# Time based analysis

In [ ]:
ddf_commits['commit_time'] = dd.to_datetime(ddf_commits.commit_time, unit='s')
ddf_commits['commit_date'] = ddf_commits.commit_time.dt.date

In [ ]:
ddf_all_commits['commit_time'] = dd.to_datetime(ddf_all_commits.commit_time, unit='s')
ddf_all_commits['commit_date'] = ddf_all_commits.commit_time.dt.date

In [ ]:
ddf_commits['commit_date'].describe().compute()

In [ ]:
ddf_all_commits['commit_date'].describe().compute()

In [ ]:
ddf_all_commits.count().compute()

In [ ]:
# ASSUMPTION: THERE ARE NO UNIQUE COMMIT HASHES FOR ACTUALLY DIFFERENT COMMITS

In [ ]:
unique_commits = ddf_commits.groupby(ddf_commits.index).first()
unique_commits.head()

In [ ]:
date_stuff = unique_commits.groupby("commit_date")[['n_added', 'n_deleted', 'n_moved', 'n_renamed', 'n_struc_changes']].sum().compute()

In [ ]:
fig = px.line(date_stuff,y='n_struc_changes')
fig.show()

In [ ]:
fig = px.line(date_stuff,y='n_added')
fig.show()

In [ ]:
fig = px.line(date_stuff,y='n_moved')
fig.show()

In [ ]:
fig = px.line(date_stuff,y='n_deleted')
fig.show()

In [ ]:
date_stuff[date_stuff['n_struc_changes'] > 400_000].head()

In [ ]:
from datetime import date


In [ ]:
commits_on_date = unique_commits[unique_commits['commit_date'] == date(2017, 9, 12)].compute()

In [ ]:
commits_on_date.sort_values('n_struc_changes', ascending=False)

In [ ]:
bla1234 = unique_commits.groupby("commit_date").n_struc_changes.count().compute()

In [ ]:
bla = ddf_all_commits.groupby('commit_date').count().compute().sort_index()

In [ ]:
all_unique_commits = ddf_all_commits.groupby(ddf_all_commits.commit_id).first()
bla1235 = all_unique_commits.groupby("commit_date").commit_time.count().compute()

In [ ]:
fig = px.line(bla1235)
fig.add_scatter(x=bla1234.index, y=bla1234,mode='lines')
fig.show()

In [ ]:
bla_perc = bla1234/bla1235
fig = px.line(bla_perc)
fig.show()

In [ ]:
fig = px.line(bla_perc.rolling(50).mean())
fig.show()

In [ ]:
ddf_commits['commit_date'].head()